## Importing Libraries

In [94]:
import pandas as pd
import networkx as nx
from neo4j import GraphDatabase
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score
from sklearn.metrics.pairwise import cosine_similarity, euclidean_distances

## Connecting to the neo4j database

In [95]:
uri = "bolt://localhost:7687"
username = "neo4j"
password = "OLIV00%%"

driver = GraphDatabase.driver(uri, auth=(username, password))

## Retrieving the graph data from Neo4j and dataset preparation

In [96]:
graph = nx.Graph()

with driver.session() as session:
    # Récupérer les nœuds avec leur nom et type
    nodes_query = "MATCH (n) RETURN ID(n) AS id, n.name AS name, n.type AS type"
    nodes_result = session.run(nodes_query)

    # Parcourir les résultats et ajouter les nœuds au graphe
    for record in nodes_result:
        node_id = record["id"]
        node_name = record["name"]
        node_type = record["type"]
        if node_id is not None:
            graph.add_node(node_id, name=node_name, type=node_type)

    # Récupérer les relations
    relations_query = "MATCH ()-[r]->() RETURN ID(startNode(r)) AS source_id, ID(endNode(r)) AS target_id"
    relations_result = session.run(relations_query)

    # Ajouter les relations entre les nœuds
    for record in relations_result:
        source_node_id = record["source_id"]
        target_node_id = record["target_id"]
        if source_node_id is not None and target_node_id is not None:
            graph.add_edge(source_node_id, target_node_id)

In [97]:
# Convert the graph to a pandas DataFrame
df = pd.DataFrame(graph.nodes(data=True), columns=["node_id", "attributes"])
df["node_type"] = df["attributes"].apply(lambda x: x.get("type"))
df["node_name"] = df["attributes"].apply(lambda x: x.get("name"))

# Drop the unnecessary 'attributes' column
df.drop(columns=["attributes"], inplace=True)

# Check if the DataFrame is empty
if df.empty:
    raise ValueError("The dataset is empty. Please check your Neo4j query and data.")

# Prepare the target variable ('link' column)
df["link"] = 1

# Split the dataset into training and testing sets
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)


In [98]:
df.head()

,node_id,node_type,node_name,link
0,0,Explanation,Static unbalance,1
1,1,Explanation,Couple unbalance,1
2,2,Explanation,Dynamic unbalance,1
3,3,Explanation,Overhung unbalance,1
4,4,Explanation,Structural looseness,1


## Generating graph embeddings using Neo4j's GDS library

In [99]:
with driver.session() as session:
    # Create a graph projection for link prediction using GDS
    session.run("CALL gds.graph.project('link_prediction_graph', '*', '*')")

In [100]:
with driver.session() as session:
    # Compute graph embeddings using Node2Vec algorithm
    session.run("""
    CALL gds.beta.node2vec.stream('link_prediction_graph', {
        embeddingDimension: 64,
        iterations: 10,
        walkLength: 20,
        inOutFactor: 1.0,
        returnFactor: 1.0
    })
    """)

## Fetch the graph embeddings 

In [101]:
# Create an empty list to store the node embeddings
node_embeddings = []

# Compute graph embeddings using Node2Vec algorithm and fetch the embeddings
with driver.session() as session:
    result = session.run("""
        CALL gds.beta.node2vec.stream('link_prediction_graph', {
            embeddingDimension: 64,
            iterations: 10,
            walkLength: 20,
            inOutFactor: 1.0,
            returnFactor: 1.0
        })
        YIELD nodeId, embedding
        RETURN nodeId, gds.util.asNode(nodeId).name AS node_name, gds.util.asNode(nodeId).type AS node_type, embedding
    """)

    # Iterate through the result and append embeddings to the list
    for record in result:
        node_id = record["nodeId"]
        node_name = record["node_name"]
        node_type = record["node_type"]
        embedding = record["embedding"]
        node_embeddings.append((node_id, node_name, node_type, embedding))

# Create a DataFrame to store the node embeddings
columns = ["node_id", "node_name", "node_type", "embedding"]
node_embeddings_df = pd.DataFrame(node_embeddings, columns=columns)

In [102]:
node_embeddings_df.head()

,node_id,node_name,node_type,embedding
0,0,Static unbalance,Explanation,"[0.0037810089997947216, -0.004961835220456123,..."
1,1,Couple unbalance,Explanation,"[0.0037796092219650745, -0.0023127037566155195..."
2,2,Dynamic unbalance,Explanation,"[0.0037095786537975073, -0.004216838628053665,..."
3,3,Overhung unbalance,Explanation,"[0.004044978879392147, -0.0016776365227997303,..."
4,4,Structural looseness,Explanation,"[0.003712308593094349, 0.0060248295776546, -0...."


In [103]:
print("Column names in train_df:", train_df.columns)
print("Column names in node_embeddings_df:", node_embeddings_df.columns)

Column names in train_df: Index(['node_id', 'node_type', 'node_name', 'link'], dtype='object')
Column names in node_embeddings_df: Index(['node_id', 'node_name', 'node_type', 'embedding'], dtype='object')


## Association

In [104]:
# Separate the nodes of type "component" and nodes of types "failure" and "ambalance"
component_nodes = node_embeddings_df[node_embeddings_df["node_type"] == "Component"]
failure_ambalance_nodes = node_embeddings_df[node_embeddings_df["node_type"].isin(["Failure", "Imbalance"])]

# Extract embeddings for each set of nodes
component_embeddings = component_nodes["embedding"].tolist()
failure_ambalance_embeddings = failure_ambalance_nodes["embedding"].tolist()

# Check if embeddings are not empty
if not component_embeddings or not failure_ambalance_embeddings:
    print("Some embeddings are empty. Check if the Node2Vec computation was successful and if nodes of required types exist.")
else:
    # Compute the similarity between the embeddings using cosine similarity
    similarity_matrix = cosine_similarity(component_embeddings, failure_ambalance_embeddings)

    # Create a DataFrame to store the similarity scores with node names as index and columns
    similarity_df = pd.DataFrame(similarity_matrix, columns=failure_ambalance_nodes["node_name"].tolist(), index=component_nodes["node_name"].tolist())

    # Display the similarity DataFrame with node names
similarity_df

,Imbalance,Structural fault,Misalignment,looseness,Bearing Fault,Gear Fault
Pump,0.168679,0.072125,0.079655,-0.037632,0.151176,0.095761
Fan,0.055122,0.312857,-0.009305,0.087522,0.097610,-0.105439
Engine,0.096407,0.195724,0.144440,0.104235,0.006602,0.099245


In [105]:
# Find the maximum similarity value for each column
max_similarities = similarity_df.max(axis=0)

# Iterate through each column and find the corresponding row(s) with the maximum value
for column in max_similarities.index:
    max_similarity = max_similarities[column]
    max_rows = similarity_df[similarity_df[column] == max_similarity].index.tolist()
    
    # Print the result
    if len(max_rows) == 1:
        print(f"{max_rows[0]} ===== lead to =====> {column}")
    else:
        rows_str = ", ".join(max_rows)
        print(f"{rows_str} ===== lead to =====> {column}")
    print()

Pump ===== lead to =====> Imbalance

Fan ===== lead to =====> Structural fault

Engine ===== lead to =====> Misalignment

Engine ===== lead to =====> looseness

Pump ===== lead to =====> Bearing Fault

Engine ===== lead to =====> Gear Fault



In [106]:
# Separate the nodes of type "component" and nodes of types "failure" and "ambalance"
component_nodes = node_embeddings_df[node_embeddings_df["node_type"] == "Component"]
failure_ambalance_nodes = node_embeddings_df[node_embeddings_df["node_type"].isin(["Failure", "Imbalance"])]

# Extract embeddings for each set of nodes
component_embeddings = component_nodes["embedding"].tolist()
failure_ambalance_embeddings = failure_ambalance_nodes["embedding"].tolist()

# Check if embeddings are not empty
if not component_embeddings or not failure_ambalance_embeddings:
    print("Some embeddings are empty. Check if the Node2Vec computation was successful and if nodes of required types exist.")
else:
    # Compute the similarity between the embeddings euclidean distance
    similarity_matrix = euclidean_distances(component_embeddings, failure_ambalance_embeddings)

    # Create a DataFrame to store the similarity scores with node names as index and columns
    similarity_df = pd.DataFrame(similarity_matrix, columns=failure_ambalance_nodes["node_name"].tolist(), index=component_nodes["node_name"].tolist())

# Display the similarity DataFrame with node names
similarity_df

,Imbalance,Structural fault,Misalignment,looseness,Bearing Fault,Gear Fault
Pump,0.046149,0.051208,0.047248,0.050725,0.046274,0.050517
Fan,0.046972,0.042416,0.047098,0.045338,0.045517,0.053502
Engine,0.046532,0.046315,0.043962,0.045525,0.048382,0.048935


In [107]:
# Find the maximum similarity value for each column
max_similarities = similarity_df.max(axis=0)

# Iterate through each column and find the corresponding row(s) with the maximum value
for column in max_similarities.index:
    max_similarity = max_similarities[column]
    max_rows = similarity_df[similarity_df[column] == max_similarity].index.tolist()
    
    # Print the result
    if len(max_rows) == 1:
        print(f"{max_rows[0]} ===== leads to =====> {column}")
    else:
        rows_str = ", ".join(max_rows)
        print(f"{rows_str} ===== leads to =====> {column}")
    print()

Fan ===== leads to =====> Imbalance

Pump ===== leads to =====> Structural fault

Pump ===== leads to =====> Misalignment

Pump ===== leads to =====> looseness

Engine ===== leads to =====> Bearing Fault

Fan ===== leads to =====> Gear Fault



In [65]:
session.close()
driver.close()